## Live Frame Extraction from Camera

#### Captures live video frames from a camera using OpenCV and saves them as individual images.

- **extract_frames_live**: This function opens the camera, captures live video frames, and saves them to the specified output folder at a given frame interval.
- **Usage**: The script can be used by providing the output folder path to save the frames and the frame interval for extraction.
- **Interactivity**: The captured frames are displayed in a window in real-time, and the user can quit the program by pressing the 'q' key.

In [2]:
import cv2
import os

def extract_frames_live(output_folder, frame_interval=1):
    # Open the camera
    cap = cv2.VideoCapture(1)
    
    if not cap.isOpened():
        print("Error: Unable to open camera.")
        return
    
    # Create the output folder
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    frame_index = 1
    
    # Loop for live video capturing and frame extraction
    while True:
        # Read a frame from the camera
        ret, frame = cap.read()
        
        # Check if frame is retrieved successfully
        if not ret:
            print("Error: Unable to capture frame.")
            break
        
        # Save frames to output folder
        if frame_index % frame_interval == 0:
            frame_path = os.path.join(output_folder, f"frame_{frame_index}.jpg")
            cv2.imwrite(frame_path, frame)
        
        # Display the captured frame
        cv2.imshow('Live Frame', frame)
        
        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        # Increment frame index
        frame_index += 1
    
    # Release the camera capture object
    cap.release()
    cv2.destroyAllWindows()

# Example usage
output_folder = "../../Component 01/Frame Extraction/Live_Extracted_Frames"
frame_interval = 5

extract_frames_live(output_folder, frame_interval)

## Live Frame Extraction from Camera with Adaptive Framerate Control & Image Compression

- **extract_frames**: This function opens the specified video capture device, captures live video frames, calculates motion levels between frames, and saves them as JPEG2000 images with a compression quality of 90.
- **Usage**: The script can be used by providing the device index of the video capture device, output folder path to save the frames, and the base frame interval for extraction.
- **Interactivity**: The captured frames are continuously displayed in a window in real-time, and the user can quit the program by pressing the 'q' key.

**Benefits:**
- Captures key moments in videos by adapting to motion.
- Reduces storage requirements and network bandwidth usage by employing lightweight JPEG2000 compression.

In [6]:
import os
import cv2
import numpy as np

def calculate_motion_level(prev_frame, frame):
    # prev_frame: The previous frame in grayscale format.
    # frame: The current frame in grayscale format.
    # Returns the average optical flow magnitude between the previous and current frames.

    # Convert frames to grayscale if not already
    if len(prev_frame.shape) > 2:
        gray_prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    else:
        gray_prev_frame = prev_frame
    if len(frame.shape) > 2:
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    else:
        gray_frame = frame

    # Calculate optical flow using Farneback method
    flow = cv2.calcOpticalFlowFarneback(gray_prev_frame, gray_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Calculate average flow magnitude
    avg_flow = np.mean(np.linalg.norm(flow, axis=2))
    return avg_flow

def extract_frames(device_index, output_folder, base_frame_interval=1):
    # Open the video capture device
    cap = cv2.VideoCapture(device_index)

    # Check if video capture device opened successfully
    if not cap.isOpened():
        print("Error: Unable to open video capture device.")
        return

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Frame index
    frame_index = 1

    # Previous frame for motion calculation
    prev_frame = None

    while True:
        # Read a frame from the video capture device
        ret, frame = cap.read()

        # Check if frame is retrieved successfully
        if not ret:
            break

        # Calculate motion level if previous frame exists
        motion_level = 0
        if prev_frame is not None:
            motion_level = calculate_motion_level(prev_frame, frame)

        # Adjust frame interval based on motion level
        frame_interval = max(base_frame_interval, int(motion_level * 5))

        # Save frame if within the interval
        if frame_index % frame_interval == 0:
            frame_path = os.path.join(output_folder, f"frame_{frame_index}.jpg")
            #cv2.imwrite(frame_path, frame, [cv2.IMWRITE_JPEG2000_COMPRESSION, 90])
            cv2.imwrite(frame_path, frame, [cv2.IMWRITE_JPEG_QUALITY, 90])

        # Update previous frame
        prev_frame = frame.copy()

        # Increment frame index
        frame_index += 1

        # Break the loop on key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture object
    cap.release()
    # Close all OpenCV windows
    cv2.destroyAllWindows()

device_index = 1
output_folder = "../../Component 01/Frame Extraction/Live_Extracted_Frames_NEW"
base_frame_interval = 5

extract_frames(device_index, output_folder, base_frame_interval)

KeyboardInterrupt: 